In [58]:
from config.settings import SRC_DIR, DATA_DIR
import os
os.chdir(SRC_DIR)

file_dir = DATA_DIR / 'invoices' / 'valid_invoices'
files = os.listdir(file_dir)
file = file_dir /files[3]
file

WindowsPath('C:/Users/ai.users/imscanCS/data/invoices/valid_invoices/PO139090 GP.pdf')

In [59]:
import pdfplumber
def get_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()

    return text

invoice_text = get_text(file)
print(invoice_text)

Catalina, Baní, República Dominicana Estado : GLOBAL G.A.P. Certificado
Tels.: (829)-341-4429/(809)-756-2359 GGN:4049929957572
E-mail: Dotfruits@hotmail.com
RNC:1-3026640-9
Fecha: 20/08/2023 Factura para Exportaciones
Consignatario: Direct Produce Supplies LTD NCF: B1600002418
Factura: 5424 Valido Hasta: 31-12-2024
Direccion:
Notificar: M&B Felixstowe LTD
Puerto de Embarque: Caucedo Naviera: Hapag-Lloyd
TOTAL KGS TOTAL KGS
CANTIDAD ENVASE DESCRIPCION TOTAL LBS BRUTAS PRECIO VALOR TOTAL US$
BRUTOS NETOS
5,544 BOXES FRESH MANGOES 47,787 24,125 22,176 5 .25 2 9,106.00
CONTENEDOR: SELLO: REG. TEMP.:
HLBU 908214-9 DGA-EXP1403247 MFXYN02XTV
HLG6904990
TEMPERATURA: 9°C
HUMEDAD 90%
VENTILACION 60 CMB/H
5,544 BOXES VALOR TOTAL US$ 2 9,106.00


In [60]:
def get_prompt(text):
    prompt = """
    You are an **invoice extractor**. Your task is to extract structured invoice details from the provided text and return **only the JSON output** in the following format:

    {
        "line_items": [
            {
                "product_code": "string",
                "description": "string",
                "quantity": "string",
                "price_per_unit": "string",
                "vat_percent": "string",
                "total_price": "string"
            }
        ],
        "total_amount": {
            "total_items": number,
            "total_tax": "string",
            "total_price": "string"
        },
        "due_date": "YYYY-MM-DD",
        "payment_date": "YYYY-MM-DD",
        "invoice_date": "YYYY-MM-DD",
        "invoice_number": "string",
        "purchase_order": "string",
        "reference_numbers": ["string"],
        "locale": "string",
        "country": "string",
        "currency": "string",
        "payment_details": {
            "iban": "string",
            "swift": "string",
            "bic": "string",
            "account_number": "string"
        },
        "vat_number": "string",
        "supplier_name": "string",
        "taxes_details": [
            {
                "rate": "string",
                "amount": "string"
            }
        ],
        "total_amount_including_taxes": "string",
        "total_net_amount_excluding_taxes": "string",
        "customer_address": "string",
        "shipping_address": "string",
        "billing_address": "string",
        "customer_company_registrations": {
            "vat_number": "string"
        },
        "customer_name": "string",
        "supplier_address": "string"
    }

    **Strict Output Rules:**
    - **Only return the data in JSON format.**
    - **Do not include any explanation, additional text, or non-JSON output.**
    - Precisely capture dates such as invoice date, due date, and payment date. You may find this in many ways like `payment terms`, `payment conditions` etc saying like `within X days`, `X days from date of INVOICE` or similar. If a due date is not directly provided but 'due in X days' is indicated, calculate it by adding X days to the invoice date.**
    - **If any value is missing, set it to `null`. Do not infer missing details or hallucinate.**
    - **Strictly follow the data types and format specified.**
    - **The output must be a valid JSON object without any additional markdown or text.**

    ### Invoice Text:

    """ + text

    return prompt

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Load the model with 4-bit quantization
def load_model(model_name="Qwen/Qwen2.5-14B"):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Enable 4-bit quantization
        bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 for computation
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",  # Automatically map to available devices
        quantization_config=bnb_config
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    return model, tokenizer

# Example: Use 4-bit quantized model to process an invoice
# Load model and tokenizer with 4-bit quantization
model, tokenizer = load_model()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [61]:
# Get the prompt for the invoice text
prompt = get_prompt(invoice_text)

# Prepare messages for the model
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant for extracting invoices."},
    {"role": "user", "content": prompt}
]

# Convert the prompt into the format the model can process
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize the input text and move it to the model's device
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate response from the model
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)

# Remove the input tokens from the generated output
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode the generated tokens into text
response = tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [62]:
import re, json
clean_json_string = re.sub(r'<\|endoftext\|>', '', response).strip()

output = json.loads(clean_json_string)
output

{'line_items': [{'product_code': None,
   'description': 'FRESH MANGOES',
   'quantity': '5,544 BOXES',
   'price_per_unit': None,
   'vat_percent': None,
   'total_price': '29,106.00'}],
 'total_amount': {'total_items': None,
  'total_tax': None,
  'total_price': '29,106.00'},
 'due_date': None,
 'payment_date': None,
 'invoice_date': '2023-08-20',
 'invoice_number': '5424',
 'purchase_order': None,
 'reference_numbers': ['B1600002418'],
 'locale': None,
 'country': None,
 'currency': 'US$',
 'payment_details': {'iban': None,
  'swift': None,
  'bic': None,
  'account_number': None},
 'vat_number': None,
 'supplier_name': None,
 'taxes_details': [{'rate': None, 'amount': None}],
 'total_amount_including_taxes': None,
 'total_net_amount_excluding_taxes': None,
 'customer_address': None,
 'shipping_address': None,
 'billing_address': None,
 'customer_company_registrations': {'vat_number': None},
 'customer_name': None,
 'supplier_address': None}

In [63]:
from json2html import json2html
from IPython.display import display, HTML

# Convert JSON to an HTML table
html_table = json2html.convert(json=output)

# Display the table
display(HTML(html_table))

In [64]:
output_folder = os.path.join(DATA_DIR / 'invoices'/ 'output_invoices')
file_name = str(file).split('\\')[-1][:-4]
import json
# Define full file path
json_file_path = os.path.join(output_folder, f"{file_name}.json")

# Save JSON file
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, indent=4)

print(f"JSON saved at: {json_file_path}")

JSON saved at: C:\Users\ai.users\imscanCS\data\invoices\output_invoices\PO139090 GP.json


In [68]:
# List all JSON files in the output folder
json_files = [f for f in os.listdir(output_folder) if f.endswith('.json')]

# Initialize an empty list to hold the merged JSON data
merged_data = []

# Read and append all JSON files
for json_file in json_files:
    file_path = os.path.join(output_folder, json_file)
    with open(file_path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            merged_data.append(data)
        except json.JSONDecodeError as e:
            print(f"Error reading {json_file}: {e}")

# Define the merged JSON output file
merged_json_path = os.path.join(output_folder, "merged_output.json")

# Save the merged data as a single JSON file
with open(merged_json_path, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4)

print(f"Merged JSON saved at: {merged_json_path}")

Merged JSON saved at: C:\Users\ai.users\imscanCS\data\invoices\output_invoices\merged_output.json
